In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import timeit
import matplotlib.pyplot as plt
from preprocess_func_new import *
from matplotlib.image import imread
%matplotlib inline


#tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
img_path = os.path.join('..','datasets','stixels')
img_path

In [ ]:
batch_size=2

In [ ]:
labels_train=pd.read_csv(os.path.join(img_path,'train','labels.csv'))
labels_train

In [ ]:
import random

idxs=list(range(len(labels_train)))
random.shuffle(idxs) 
print(idxs)

batches_idx = [idxs[x:x+batch_size] for x in range(0, len(idxs), batch_size)]
print(batches_idx)



In [ ]:
labels_train.loc[batches_idx[0]]

In [ ]:
def print_progress(count, total):
    # Percentage completion.
    pct_complete = float(count) / total

    # Status-message.
    # Note the \r which means the line should overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [ ]:
def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def convert(image_paths, labels, out_path):
    # Args:
    # image_paths   List of file-paths for the images.
    # labels        Class-labels for the images.
    # out_path      File-path for the TFRecords output file.
    print('')
    print("Converting: " + out_path)
    
    # Number of images. Used when printing the progress.
    num_images = len(image_paths)
    
    # Open a TFRecordWriter for the output-file.
    with tf.python_io.TFRecordWriter(out_path) as writer:
        
        # Iterate over all the image-paths and class-labels.
        for i, (path, label) in enumerate(zip(image_paths, labels)):
            # Print the percentage-progress.
            print_progress(count=i, total=num_images-1)
            
            with open(path, 'rb') as f:
                img_raw = f.read()
           
            # Create a dict with the data we want to save in the
            # TFRecords file. You can add more relevant data here.
            data = \
                {
                    'image': wrap_bytes(img_raw),
                    'label': wrap_int64(label)
                } 

            # Wrap the data as TensorFlow Features.
            feature = tf.train.Features(feature=data)

            # Wrap again as a TensorFlow Example.
            example = tf.train.Example(features=feature)

            # Serialize the data.
            serialized = example.SerializeToString()
            
            # Write the serialized data to the TFRecords file.
            writer.write(serialized)

In [ ]:
import sys
for batch in batches_idx:
    batch_labels=labels.loc[batch]
    batch_names_list=list(batch_labels['Name'])
    batch_labels=np.array(batch_labels['label'])
    batch_image_paths_train=[]
    for name in batch_names_list:
        batch_image_paths_train.append(os.path.join(img_path, 'train', name))#+'.png')) #maybe no need to add '.png'
    batch_path_tfrecords_train = os.path.join(img_path, 'train', "train"+str(batch[0])+".tfrecords") #maybe use enum
    convert(image_paths=batch_image_paths_train,
        labels=batch_labels,
        out_path=batch_path_tfrecords_train)

In [ ]:
#VAL:

In [ ]:
labels_val=pd.read_csv(os.path.join(img_path,'val','labels.csv'))
labels_val

In [ ]:
val_names_list=list(labels_val['Name'])
val_labels=np.array(labels_val['label'])
image_paths_val=[]
for name in val_names_list:
    image_paths_val.append(os.path.join(img_path, 'val', name))#+'.png')) #maybe no need to add '.png'
path_tfrecords_val = os.path.join(img_path, 'val', "val.tfrecords")
convert(image_paths=image_paths_val,
        labels=val_labels,
        out_path=path_tfrecords_val)

In [ ]:
#TEST:

In [ ]:
labels_test=pd.read_csv(os.path.join(img_path,'test','labels.csv'))
labels_test

In [ ]:
test_names_list=list(labels_test['Name'])
test_labels=np.array(labels_test['label'])
image_paths_test=[]
for name in test_names_list:
    image_paths_test.append(os.path.join(img_path, 'test', name))#+'.png')) #maybe no need to add '.png'
path_tfrecords_test = os.path.join(img_path, 'test', "test.tfrecords")
convert(image_paths=image_paths_test,
        labels=test_labels,
        out_path=path_tfrecords_test)

In [ ]:
################################################################################

In [ ]:
########now part 2: decode and train#######

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7922305115373106610
]


In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import timeit
import matplotlib.pyplot as plt
#from preprocess_func_new import *
from matplotlib.image import imread

%matplotlib inline


#tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
img_path = os.path.join('..','datasets','stixels')
img_path

'../datasets/stixels'

In [5]:
batch_size=2

In [14]:
labels_test=pd.read_csv(os.path.join(img_path,'test','labels.csv'))
test_names_list=list(labels_test['Name'])
image_paths_test=[]
for name in test_names_list:
    image_paths_test.append(os.path.join(img_path, 'test', name))#+'.png')) #maybe no need to add '.png'


In [45]:
path_tfrecords_train_lst=[]
path_tfrecords_train = os.path.join(img_path, 'train')
for root, dirs, files in os.walk(path_tfrecords_train):
    for file in files:
        if '.tfrecord' in file:
            path_tfrecords_train_lst.append(os.path.join(path_tfrecords_train,file))
print(path_tfrecords_train_lst)


['../datasets/stixels/train/train0.tfrecords', '../datasets/stixels/train/train1.tfrecords']


In [31]:
H=370
W=24
C=3
img_size_flat = H*W
img_shape = (H,W, C)
num_classes = 47

In [32]:
def parse(serialized):
    # Define a dict with the data-names and types we expect to
    # find in the TFRecords file.
    # It is a bit awkward that this needs to be specified again,
    # because it could have been written in the header of the
    # TFRecords file instead.
    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        }

    # Parse the serialized data so we get a dict with our data.
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)

    # Get the image as raw bytes.
    image_raw = parsed_example['image']

    # Decode the raw bytes so it becomes a tensor with type.
    #######image = tf.decode_raw(image_raw, tf.int32) ####
    image = tf.image.decode_png(image_raw, channels=3)
    #image = tf.cast(image, tf.int32)
    print('image:')
    print(image)
    # The type is now uint8 but we need it to be float.
    
    image = tf.cast(image, tf.float32) ####
    
    # Get the label associated with the image.
    label = parsed_example['label']

    # The image and label are now correct TensorFlow types.
    return image, label

In [33]:
def input_fn(filenames, train, batch_size=batch_size, buffer_size=2048):
    # Args:
    # filenames:   Filenames for the TFRecords files.
    # train:       Boolean whether training (True) or testing (False).
    # batch_size:  Return batches of this size.
    # buffer_size: Read buffers of this size. The random shuffling
    #              is done on the buffer, so it must be big enough.

    # Create a TensorFlow Dataset-object which has functionality
    # for reading and shuffling data from TFRecords files.
    dataset = tf.data.TFRecordDataset(filenames=filenames)

    # Parse the serialized data in the TFRecords files.
    # This returns TensorFlow tensors for the image and labels.
    dataset = dataset.map(parse)

    if train:
        # If training then read a buffer of the given size and
        # randomly shuffle it.
        dataset = dataset.shuffle(buffer_size=buffer_size)

        # Allow infinite reading of the data.
        num_repeat = None
    else:
        # If testing then don't shuffle the data.
        
        # Only go through the data once.
        num_repeat = 1

    # Repeat the dataset the given number of times.
    dataset = dataset.repeat(num_repeat)
    
    # Get a batch of data with the given size.
    dataset = dataset.batch(batch_size)

    # Create an iterator for the dataset and the above modifications.
    iterator = dataset.make_one_shot_iterator()

    # Get the next batch of images and labels.
    images_batch, labels_batch = iterator.get_next()

    # The input-function must return a dict wrapping the images.
    x = {'image': images_batch}
    y = labels_batch
    
    return x, y

In [34]:
def train_input_fn():
    return input_fn(filenames=path_tfrecords_train_lst, train=True)

In [35]:
def val_input_fn():
    return input_fn(filenames=path_tfrecords_val, train=False)

In [36]:
def test_input_fn():
    return input_fn(filenames=path_tfrecords_test, train=False)

In [37]:
def load_images(image_paths):
    # Load the images from disk.
    images = [imread(path) for path in image_paths]
    # Convert to a numpy array and return it.
    return np.asarray(images)


In [70]:
some_num=1
some_images = load_images(image_paths=image_paths_test[0:some_num+1])
some_images_cls = np.array(labels_test['label'])
print(some_images.shape)

(2, 370, 24, 3)


In [71]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": some_images.astype(np.float32)},
    num_epochs=1,
    shuffle=False)

In [40]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "image" in the input-function.
    x = features["image"]
    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1, W, H, C])    
    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=64, kernel_size=(11,5),
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(8,4), strides=1)
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=200, kernel_size=(5,3),
                           padding='same', activation=tf.nn.relu) #200
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(4,3), strides=1)    

    # Flatten to a 2-rank tensor.
    #net = tf.contrib.layers.flatten(net)
    # Eventually this should be replaced with:
    net = tf.layers.flatten(net)

    # First fully-connected / dense layer.
    # This uses the ReLU activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=206, activation=tf.nn.relu)     #1024
    
    # Second fully-connected / dense layer
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=206, activation=tf.nn.relu)   #2048
    
   
    # This is the last layer so it does not use an activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc3',
                          units=47)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        metrics = \
        {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

In [41]:
params = {"learning_rate": 1e-4}

In [42]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir="./checkpoints_tutorial18-3/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_train_distribute': None, '_session_config': None, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_save_summary_steps': 100, '_master': '', '_num_ps_replicas': 0, '_is_chief': True, '_tf_random_seed': None, '_model_dir': './checkpoints_tutorial18-3/', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f49fc8a29e8>, '_save_checkpoints_secs': 600, '_service': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_evaluation_master': ''}


In [43]:
tf.logging.set_verbosity(tf.logging.INFO)

In [47]:
model.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_tutorial18-3/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./checkpoints_tutorial18-3/model.ckpt.
INFO:tensorflow:loss = 1141.1287, step = 2
INFO:tensorflow:Saving checkpoints for 11 into ./checkpoints_tutorial18-3/model.ckpt.
INFO:tensorflow:Loss for final step: 238.28821.


image:
Tensor("DecodePng:0", shape=(?, ?, 3), dtype=uint8)
x in net:
Tensor("IteratorGetNext:0", shape=(?, ?, ?, 3), dtype=float32, device=/device:CPU:0)
net in net:
Tensor("Reshape:0", shape=(?, 24, 370, 3), dtype=float32)


In [ ]:
#####DONE TRAIN######

In [ ]:
# VAL 

In [50]:
path_tfrecords_val=os.path.join(img_path,'val','val.tfrecords')

In [51]:
val_result = model.evaluate(input_fn=val_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-08-14:17:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_tutorial18-3/model.ckpt-11
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-08-14:17:24
INFO:tensorflow:Saving dict for global step 11: accuracy = 0.75, global_step = 11, loss = 68.71109


image:
Tensor("DecodePng:0", shape=(?, ?, 3), dtype=uint8)
x in net:
Tensor("IteratorGetNext:0", shape=(?, ?, ?, 3), dtype=float32, device=/device:CPU:0)
net in net:
Tensor("Reshape:0", shape=(?, 24, 370, 3), dtype=float32)


In [52]:
val_result

{'accuracy': 0.75, 'global_step': 11, 'loss': 68.71109}

In [53]:
print("Classification val accuracy: {0:.2%}".format(val_result["accuracy"]))

Classification val accuracy: 75.00%


In [ ]:
###test###

In [57]:
path_tfrecords_test=os.path.join(img_path,'test','test.tfrecords')

In [59]:
test_result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-08-14:26:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_tutorial18-3/model.ckpt-11
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-08-14:26:40
INFO:tensorflow:Saving dict for global step 11: accuracy = 0.75, global_step = 11, loss = 68.71109


image:
Tensor("DecodePng:0", shape=(?, ?, 3), dtype=uint8)
x in net:
Tensor("IteratorGetNext:0", shape=(?, ?, ?, 3), dtype=float32, device=/device:CPU:0)
net in net:
Tensor("Reshape:0", shape=(?, 24, 370, 3), dtype=float32)


In [61]:
test_result

{'accuracy': 0.75, 'global_step': 11, 'loss': 68.71109}

In [62]:
print("Classification test accuracy: {0:.2%}".format(test_result["accuracy"]))

Classification test accuracy: 75.00%


In [ ]:
###pred###

In [72]:
predictions = model.predict(input_fn=predict_input_fn) ###FIX THIS!!!! TODO

In [75]:
cls_pred = np.array(list(predictions))
cls_pred #FIX THIS!!!!!! TODO

array([], dtype=float64)